## Imports

In [65]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 

## Data Transforming

In [66]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [67]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,1,225,308751
798459,0,6,0,0,1,0,0,0,3,0,0,0,1,690,308750
798460,0,9,0,0,2,0,0,0,3,0,0,0,1,215,308771
798461,0,16,0,0,2,0,0,0,10,0,0,0,1,219,308776


In [68]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [69]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [70]:
# c=data.subdomain_encoded.nunique()
# data.subdomain_encoded.nunique()

In [71]:
d=a+b
d

310015

In [72]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146


In [73]:
y=data['status']
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)

In [74]:
X.shape

(798463, 266)

In [75]:
X=X.toarray()
print(X.shape)
X=X.reshape(X.shape[0],14,19)

(798463, 266)


In [76]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=54)
X_train.shape


(558924, 14, 19)

In [77]:
# X_train = pad_sequences(X_train, padding='post', truncating='post')
# X_test = pad_sequences(X_test, padding='post', truncating='post')

## CNN

In [78]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [79]:
cnn  = Sequential()
cnn.add(Conv1D(128,(5), input_shape=(14,19), activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
# cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
cnn.add(Dense(1,activation ='sigmoid'))
# maybe consider increaseing dropout as this model has a high starting accuracy so combat overfitting
# look into handling the categorical features and concatenating with the numeric features to see if that makes a difference for embedding layers

In [80]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [81]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around 6 minutes an epoch when split 
# reached around 88

Epoch 1/30
17467/17467 - 83s - 5ms/step - accuracy: 0.8993 - loss: 0.2461 - precision: 0.8861 - recall: 0.9314 - val_accuracy: 0.9179 - val_loss: 0.2057 - val_precision: 0.9026 - val_recall: 0.9488
Epoch 2/30
17467/17467 - 100s - 6ms/step - accuracy: 0.9163 - loss: 0.2083 - precision: 0.9046 - recall: 0.9429 - val_accuracy: 0.9236 - val_loss: 0.1912 - val_precision: 0.9127 - val_recall: 0.9476
Epoch 3/30
17467/17467 - 81s - 5ms/step - accuracy: 0.9209 - loss: 0.1972 - precision: 0.9097 - recall: 0.9460 - val_accuracy: 0.9255 - val_loss: 0.1850 - val_precision: 0.9118 - val_recall: 0.9529
Epoch 4/30
17467/17467 - 89s - 5ms/step - accuracy: 0.9238 - loss: 0.1905 - precision: 0.9127 - recall: 0.9481 - val_accuracy: 0.9269 - val_loss: 0.1813 - val_precision: 0.9136 - val_recall: 0.9534
Epoch 5/30
17467/17467 - 132s - 8ms/step - accuracy: 0.9258 - loss: 0.1859 - precision: 0.9152 - recall: 0.9492 - val_accuracy: 0.9280 - val_loss: 0.1789 - val_precision: 0.9167 - val_recall: 0.9518
Epoch 6/